In [1]:
import feedparser
import pandas as pd
import numpy as np
import re
from datetime import datetime
import spacy

In [2]:
#Solo ejecutar la primera vez
corpus = pd.DataFrame(columns = ['Source','Title','Content','Section','URL','Date'])
corpus.head()

,Unnamed: 0,Source,Title,Content,Section,URL,Date
0,0,Expansion,CFE Telecom perfilar a nuevo CEO para impulsar...,"Según dos fuente con conocimiento del tema , a...",tecnologia,https://expansion.mx/tecnologia/2024/09/22/cfe...,22/09/2024
1,1,Expansion,Cómo Caliente pasar de anunciar apuesta a tran...,La compañía ha entrar a la ‘ guerra del stream...,tecnologia,https://expansion.mx/tecnologia/2024/09/22/cal...,22/09/2024
2,2,Expansion,Esta es la audiencia que quiere conquistar el ...,Apple es consciente de que muchos usuario de i...,tecnologia,https://expansion.mx/tecnologia/2024/09/20/iph...,20/09/2024
3,3,Expansion,Microsoft usar energía nuclear para alimentar ...,El acuerdo entre Microsoft y Constellation Ene...,tecnologia,https://expansion.mx/tecnologia/2024/09/20/mic...,20/09/2024
4,4,Expansion,La ONU formular siete recomendación para regul...,La ONU crear el año pasado un órgano consultiv...,tecnologia,https://expansion.mx/tecnologia/2024/09/20/la-...,20/09/2024


#### RSS de la Jornada

In [3]:
deportes_jornada = feedparser.parse('https://www.jornada.com.mx/rss/deportes.xml?v=1')
economia_jornada = feedparser.parse('https://www.jornada.com.mx/rss/economia.xml?v=1')
cultura_jornada = feedparser.parse('https://www.jornada.com.mx/rss/cultura.xml?v=1')
ciencia_jornada = feedparser.parse('https://www.jornada.com.mx/rss/ciencias.xml?v=1')



#### RSS de Expansión

In [4]:
tecnologia_expansion = feedparser.parse('https://expansion.mx/rss/tecnologia')
economia_expansion = feedparser.parse('https://expansion.mx/rss/economia')

In [5]:
def jornada (jornada_parser):
    # Crear una lista para almacenar los datos
    data_list = []

    for entry in jornada_parser.entries:

        # Extraer la sección a partir del enlace
        link = entry.title_detail.base
        match = re.search(r'/rss/(\w+)\.xml', link)
        entry_link = match.group(1) if match else 'Unknown'

        # Convertir la fecha de publicación al formato 01/01/2000
        fecha = deportes_jornada.entries[0].published
        fecha_obj = datetime.strptime(fecha, '%a, %d %b %Y %H:%M:%S GMT')
        fecha_format = fecha_obj.strftime('%d/%m/%Y')

                # Crear un diccionario con los datos
        row_data = {
            'Source': 'Jornada',
            'Title': entry.title,
            'Content': entry.summary,
            'Section': entry_link,
            'URL': entry.links[0].href,
            'Date': fecha_format
        }
        # Checamos que no exista esa noticia en el corpus
        if entry.title not in corpus['Title']:
            # Añadir el diccionario a la lista
            data_list.append(row_data)
        
    # Convertir la lista de diccionarios en un DataFrame
    df = pd.DataFrame(data_list, columns=['Source', 'Title', 'Content', 'Section', 'URL', 'Date'])
    
    return df
        

In [6]:
def expansion(Expansion_Parser):
    # Crear una lista para almacenar los datos
    data_list = []

    for entry in Expansion_Parser.entries:

        # Extraer la sección a partir del enlace
        link = entry.title_detail.base 
        match = re.search(r'rss\/(\w+)', link)
        entry_link = match.group(1) if match else 'Unknown'

        # Convertir la fecha de publicación al formato 01/01/2000
        fecha = entry.published
        fecha_obj = datetime.strptime(fecha, '%a, %d %b %Y %H:%M:%S GMT')
        fecha_format = fecha_obj.strftime('%d/%m/%Y')

        # Crear un diccionario con los datos
        row_data = {
            'Source': 'Expansion',
            'Title': entry.title,
            'Content': entry.summary,
            'Section': entry_link,
            'URL': entry.link,
            'Date': fecha_format
        }
        
        # Checamos que no exista esa noticia en el corpus
        if entry.title not in corpus['Title']:
            # Añadir el diccionario a la lista
            data_list.append(row_data)
    
    # Convertir la lista de diccionarios en un DataFrame
    df = pd.DataFrame(data_list, columns=['Source', 'Title', 'Content', 'Section', 'URL', 'Date'])
    
    return df



    

In [7]:
tecnologia_exp_df = expansion(tecnologia_expansion)
tecnologia_exp_df.head()


,Source,Title,Content,Section,URL,Date
0,Expansion,Netflix y Prime Video de Amazon lideran quejas...,"A agosto de este año, ambas empresas recibiero...",tecnologia,https://expansion.mx/tecnologia/2024/09/23/pla...,23/09/2024
1,Expansion,¿Qué son y cómo funcionan las nuevas categoría...,Con la nueva organización del directorio de Ca...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/nue...,23/09/2024
2,Expansion,"Intel, el nuevo objeto del deseo en la industr...",La empresa pionera de la industria de los chip...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/int...,23/09/2024
3,Expansion,"India, el país que quiere ser potencia en la i...",Estados Unidos y la India anunciaron una alian...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/ind...,23/09/2024
4,Expansion,AMLO busca convertir a CFE Telecom en una empr...,La iniciativa de reforma quiere que CFE TEIT s...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/aml...,23/09/2024


In [8]:
economia_exp_df = expansion(economia_expansion)
economia_exp_df.head()

,Source,Title,Content,Section,URL,Date
0,Expansion,Denuncian a Visa por monopolio en Estados Unidos,El Departamento de Justicia de Estados Unidos ...,economia,https://expansion.mx/economia/2024/09/23/denun...,24/09/2024
1,Expansion,Empresas de EU reactivarán anuncios de inversi...,Tras detener un anuncio importante de inversió...,economia,https://expansion.mx/economia/2024/09/23/empre...,23/09/2024
2,Expansion,El endeudamiento de mexicanos en el exterior r...,El Banco de México (Banxico) dio a conocer que...,economia,https://expansion.mx/economia/2024/09/23/deuda...,23/09/2024
3,Expansion,Las auditorías dan ingresos por 3.5 billones d...,"De estos recursos, 1.426 billones de pesos se ...",economia,https://expansion.mx/economia/2024/09/23/audit...,23/09/2024
4,Expansion,Economía mexicana crece 0.6% en julio,Las actividades primarias encabezaron el creci...,economia,https://expansion.mx/economia/2024/09/23/econo...,23/09/2024


In [9]:
ciencia_jor_df = jornada(ciencia_jornada)
ciencia_jor_df.head()

,Source,Title,Content,Section,URL,Date


In [10]:
cultura_jor_df = jornada(cultura_jornada)
cultura_jor_df.head()

,Source,Title,Content,Section,URL,Date
0,Jornada,De Anahit a María sale de Gran Bretaña a Armenia,"La exhibición Diosa Madre: De Anahit a María, ...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
1,Jornada,La obra Nombres de combate reúne en el foro La...,"Hay historias que merecen ser contadas, no sól...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
2,Jornada,Hermann Bellinghausen: Stravinski y la vendedo...,La segunda parte de los años 40 del siglo X...,cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
3,Jornada,"Murió Fredric Jameson, crítico del capitalismo...","Fredric Jameson (1934), crítico, teórico y cat...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
4,Jornada,López Obrador inaugura en Veracruz el Centro C...,"Veracruz, Ver., El presidente Andrés Manuel Ló...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024


In [11]:
economia_jor_df = jornada(economia_jornada)
economia_jor_df.head()

,Source,Title,Content,Section,URL,Date
0,Jornada,"Afores ganan $ 8,359 mdp por gestionar ahorro ...",Las ganancias que obtuvieron las Administrador...,economia,https://www.jornada.com.mx/2024/09/23/economia...,23/09/2024
1,Jornada,Enrique Galván Ochoa: Dinero,La presidenta de la República que tomará po...,economia,https://www.jornada.com.mx/2024/09/23/opinion/...,23/09/2024
2,Jornada,León Bendesky: El ejemplo,El escritor inglés H.G. Wells apuntó que la Hi...,economia,https://www.jornada.com.mx/2024/09/23/economia...,23/09/2024
3,Jornada,Mejoraron condiciones económicas de millones d...,La nueva política económica y social que puso ...,economia,https://www.jornada.com.mx/2024/09/23/economia...,23/09/2024
4,Jornada,Concentra la manufactura el flujo de inversión...,"Entre enero de 2023 y septiembre de 2024, el s...",economia,https://www.jornada.com.mx/2024/09/23/economia...,23/09/2024


In [12]:
deportes_jor_df = jornada(deportes_jornada)
deportes_jor_df.head()


,Source,Title,Content,Section,URL,Date
0,Jornada,Dos olímpicos conquistan Zúrich,Los campeones olímpicos Remco Evenepoel (image...,deportes,https://www.jornada.com.mx/2024/09/23/deportes...,23/09/2024
1,Jornada,Charlyn Corral logra doblete y mantiene el inv...,"En un duelo vertiginoso, Charlyn Corral firmó ...",deportes,https://www.jornada.com.mx/2024/09/23/deportes...,23/09/2024
2,Jornada,"Pumas sufre de más, pero rescata victoria 1-0 ...","Pumas sufrió de más, pero rescató este domingo...",deportes,https://www.jornada.com.mx/2024/09/23/deportes...,23/09/2024
3,Jornada,"Alcaraz da el triunfo decisivo para Europa, ca...",París. Europa conquistó ayer la Copa Laver de ...,deportes,https://www.jornada.com.mx/2024/09/23/deportes...,23/09/2024
4,Jornada,Jefes continúan invictos; superan 22-17 a Halc...,Atlanta. Patrick Mahomes lanzó un par de pases...,deportes,https://www.jornada.com.mx/2024/09/23/deportes...,23/09/2024


In [13]:
corpus = pd.concat([tecnologia_exp_df,economia_exp_df, deportes_jor_df, economia_jor_df, ciencia_jor_df, cultura_jor_df], axis = 0)
corpus

,Source,Title,Content,Section,URL,Date
0,Expansion,Netflix y Prime Video de Amazon lideran quejas...,"A agosto de este año, ambas empresas recibiero...",tecnologia,https://expansion.mx/tecnologia/2024/09/23/pla...,23/09/2024
1,Expansion,¿Qué son y cómo funcionan las nuevas categoría...,Con la nueva organización del directorio de Ca...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/nue...,23/09/2024
2,Expansion,"Intel, el nuevo objeto del deseo en la industr...",La empresa pionera de la industria de los chip...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/int...,23/09/2024
3,Expansion,"India, el país que quiere ser potencia en la i...",Estados Unidos y la India anunciaron una alian...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/ind...,23/09/2024
4,Expansion,AMLO busca convertir a CFE Telecom en una empr...,La iniciativa de reforma quiere que CFE TEIT s...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/aml...,23/09/2024
...,...,...,...,...,...,...
1,Jornada,La obra Nombres de combate reúne en el foro La...,"Hay historias que merecen ser contadas, no sól...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
2,Jornada,Hermann Bellinghausen: Stravinski y la vendedo...,La segunda parte de los años 40 del siglo X...,cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
3,Jornada,"Murió Fredric Jameson, crítico del capitalismo...","Fredric Jameson (1934), crítico, teórico y cat...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
4,Jornada,López Obrador inaugura en Veracruz el Centro C...,"Veracruz, Ver., El presidente Andrés Manuel Ló...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024


In [14]:
corpus.drop_duplicates(subset = 'Title', inplace = True)

In [15]:

# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')

# Crear listas temporales para almacenar los resultados procesados
new_titles = []
new_contents = []

# Iterar sobre cada fila del DataFrame
for i, row in corpus.iterrows():
    doc_title = nlp(row['Title'])
    doc_content = nlp(row['Content'])
    
    # Procesar el título: reemplazar tokens por sus lemas
    new_title = " ".join([token.lemma_ if not token.is_stop else token.text for token in doc_title])
    new_titles.append(new_title)
    
    # Procesar el contenido: reemplazar tokens por sus lemas
    new_content = " ".join([token.lemma_ if not token.is_stop else token.text for token in doc_content])
    new_contents.append(new_content)

# Asignar las listas modificadas al DataFrame
corpus['Title'] = new_titles
corpus['Content'] = new_contents


-articulo
-conjuncion
-pronombre

In [16]:
corpus

,Source,Title,Content,Section,URL,Date
0,Expansion,netflix y Prime Video de Amazon liderar queja ...,"A agosto de este año , ambos empresa recibir i...",tecnologia,https://expansion.mx/tecnologia/2024/09/23/pla...,23/09/2024
1,Expansion,¿ Qué son y cómo funcionar las nuevas categorí...,Con la nueva organización del directorio de Ca...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/nue...,23/09/2024
2,Expansion,"Intel , el nuevo objeto del deseo en la indust...",La empresa pionera de la industria de los chip...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/int...,23/09/2024
3,Expansion,"India , el país que quiere ser potencia en la ...",Estados Unidos y la India anunciar una alianza...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/ind...,23/09/2024
4,Expansion,AMLO buscar convertir a CFE Telecom en una emp...,La iniciativa de reforma quiere que CFE TEIT s...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/aml...,23/09/2024
...,...,...,...,...,...,...
1,Jornada,La obra Nombres de combate reúnir en el foro L...,"Hay historia que merecer ser contar , no sólo ...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
2,Jornada,Hermann Bellinghausen : Stravinski y la vended...,La segunda parte de los año 40 del siglo x...,cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
3,Jornada,"Murió Fredric Jameson , crítico del capitalism...","Fredric Jameson ( 1934 ) , crítico , teórico y...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
4,Jornada,López Obrador inaugurar en Veracruz el Centro ...,"Veracruz , Ver . , El presidente Andrés Manuel...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024


In [17]:
corpus = corpus.reset_index(drop=True)
corpus.to_csv('corpus1.csv')

In [18]:
corpus

,Source,Title,Content,Section,URL,Date
0,Expansion,netflix y Prime Video de Amazon liderar queja ...,"A agosto de este año , ambos empresa recibir i...",tecnologia,https://expansion.mx/tecnologia/2024/09/23/pla...,23/09/2024
1,Expansion,¿ Qué son y cómo funcionar las nuevas categorí...,Con la nueva organización del directorio de Ca...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/nue...,23/09/2024
2,Expansion,"Intel , el nuevo objeto del deseo en la indust...",La empresa pionera de la industria de los chip...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/int...,23/09/2024
3,Expansion,"India , el país que quiere ser potencia en la ...",Estados Unidos y la India anunciar una alianza...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/ind...,23/09/2024
4,Expansion,AMLO buscar convertir a CFE Telecom en una emp...,La iniciativa de reforma quiere que CFE TEIT s...,tecnologia,https://expansion.mx/tecnologia/2024/09/23/aml...,23/09/2024
...,...,...,...,...,...,...
119,Jornada,La obra Nombres de combate reúnir en el foro L...,"Hay historia que merecer ser contar , no sólo ...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
120,Jornada,Hermann Bellinghausen : Stravinski y la vended...,La segunda parte de los año 40 del siglo x...,cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
121,Jornada,"Murió Fredric Jameson , crítico del capitalism...","Fredric Jameson ( 1934 ) , crítico , teórico y...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
122,Jornada,López Obrador inaugurar en Veracruz el Centro ...,"Veracruz , Ver . , El presidente Andrés Manuel...",cultura,https://www.jornada.com.mx/2024/09/23/cultura/...,23/09/2024
